In [1]:
%load_ext blackcellmagic

In [2]:
import pathlib

import numpy as np
import plonk

from bokeh.io import output_notebook, show
from bokeh.palettes import Inferno11
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [12]:
directory = pathlib.Path('~/runs/multigrain/dustywave/Epstein-f=0.1-equal').expanduser()
sim = plonk.load_sim(prefix='dustywave', directory=directory)

In [13]:
# Plot first n_snaps snaps
n_snaps = 10
snaps = sim.snaps[::n_snaps]

# Plot every n_particle particles
n_particles = 1
s = slice(None, None, n_particles)

x_range = (-0.6, 0.6)
y_range = (-10e-4, 10e-4)

fig = figure(x_range=x_range, y_range=y_range)
for snap, color in zip(snaps, Inferno11):
    fig.scatter(
        snap["x"][s], snap["vx"][s], line_color=color, fill_color=color, size=5
    )
show(fig)